In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.pardir)

In [7]:
from os.path import join
import joblib
import json
from glob import glob

In [73]:
import pdb

In [61]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [23]:
from hmmlearn.hmm import GaussianHMM

In [69]:
from hmm.hmm_utils import tempered_sampling
from utils.utils import load_netG, overlap_decode

In [29]:
from reconstruction_error.fidelity import evaluate_generative_model

In [30]:
from data.dataset import bird_dataset_single_hdf

In [40]:
opts = dict(modelpath = '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/',
           datapath = '/home/songbird/datapartition/all_birds.hdf',
           birdname = 'p3r16',
            cuda = 'True',
            alpha = 0.95,
            beta = 0.96,
            nu = 1.,
            nrnn = 100,
            nrnnlayers = 1,
            dropout = 0.0,
            batch_size = 64,
            nepochs = 30,
            lr = 1e-3,
            l2 = 0.0,
            beta1 = 0.5,
            log_every = 100,
            checkpoint_models = False,
            
            nembed = 3,
            nlin = 50,
            kneighbors = 10,
            max_length=100
           )

In [ ]:
# which days were trained for this bird and model type
day_folders = sorted(glob(join(opts['modelpath'], 'day*')))

# make bird dataset
dataset = bird_dataset_single_hdf(opts['datapath'], opts['birdname'])

# model opts
with open(join(opts['modelpath'], 'opts.json'), 'rb') as f:
    model_opts = json.load(f)

    

In [113]:
day_folders[-1]

'/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_9'

In [ ]:
# find the day index from the name
dayname = os.path.split(day)[-1]
dayidx = int(dayname.split('_')[-1])
print('..... working on day %s .....'%(dayname))           

# get real data for this day 
X = dataset.get(dayidx, nsamps=-1) 

In [14]:
len(X)

511

In [19]:
# find which netG to use
netGfilepath = sorted(glob(join(day, 'netG_*')))[-1]

netG = load_netG(netGfilepath, model_opts['nz'], model_opts['ngf'], model_opts['nc'],
                opts['cuda'], resnet=True)


In [58]:
from reconstruction_error.fidelity import evaluate_generative_model, learn_embedding

In [ ]:
hmm_models = glob(join(day, 'hmm_*'))

#for k in range(len(hmm_models)):
k = 0

# load hmm model
hmm = joblib.load(join(day, hmm_models[k], 'model_day_'+str(dayidx)+'.pkl'))
hmm = hmm['model']

# first train embedding on real data
if embedder is None:
    embedder = learn_embedding(X, opts)
    embedder.eval()

metrics = evaluate_generative_model(netG, hmm, X, opts, embedder)

avg_precision = metrics[0].cpu().numpy().mean()
avg_recall =  metrics[1].cpu().numpy().mean()
avg_authenticity =  metrics[2].cpu().numpy().mean()

print(f"..... day {dayname}, hmm states {hmm.transmat_.shape[0]} avg precision {avg_precision} .....")
print(f"..... day {dayname}, hmm states {hmm.transmat_.shape[0]} avg recall {avg_recall} .....")
print(f"..... day {dayname}, hmm states {hmm.transmat_.shape[0]} avg authenticity {avg_authenticity} .....")



In [117]:
hmm_models = sorted(glob(join(day, 'hmm_*')))
hmm_models

['/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_10',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_12',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_15',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_20',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_25',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_30',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_40',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_5',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_50',
 '/home/songbird/datapartition/mdgan_output/daily_gan/p3r16_nz12/day_10/hmm_hiddensize_60']

In [67]:
data = X.copy()
N = len(data)

In [70]:
with torch.no_grad():

    print('..... generating fake samples .....')
    timesteps = [seq.shape[1]//16 for seq in data]
    sample_seqs = generate_samples(netG, hmm, N, 1., timesteps, opts['cuda'])

    # embed all
    print('..... embedding fakes and real samples .....')
    fake_pts = embed_list_of_sequences(embedder, sample_seqs, cuda=opts['cuda'])
    real_pts = embed_list_of_sequences(embedder, data, cuda=opts['cuda'])

..... generating fake samples .....
..... embedding fakes and real samples .....


In [79]:

# nearest neighbor
print('..... learning nearest neighbors for all reals .....')
nearest_neighbors = knn(real_pts, opts['kneighbors'])


..... learning nearest neighbors for all reals .....


In [110]:
 # generate fake sample sequences
with torch.no_grad():

    # radius of the real Ball 
    center_r = torch.zeros(opts['nembed'])
    if opts['cuda']:
        center_r = center_r.cuda()

    r_alpha = compute_quantile_radius(real_pts, center_r, opts['alpha'])
    print('..... alpha radius for real ball is %.4f .....'%(float(r_alpha)))

    # sample wise metrics
    # precision
    print('..... computing precision for all fake samples .....')
    P = np.array([precision_classifier(fake_pts[i], center_r, r_alpha) for i in range(N)])

    # recall
    # make center of fake Ball the average fake data
    c_g = fake_pts.mean(dim=0)
    # get ball B radius for ball of generated samples
    r_beta = compute_quantile_radius(fake_pts, c_g, opts['beta'])
    print('..... beta radius for fake ball is %.4f .....'%(r_beta))

    print('..... computing recall for all fake samples .....')
    R = np.array([recall_classifier(y, fake_pts, nearest_neighbors, r_beta, opts['kneighbors'],
                      beta=opts['beta']) for y in real_pts])

    # authenticity 
    print('..... computing authenticity for all fake samples .....')
    A = np.array([authenticity_classifier(y, real_pts) for y in fake_pts]).mean()
    
    D, C = density_and_coverage(fake_pts, real_pts, opts['kneighbors'])

..... alpha radius for real ball is 0.0152 .....
..... computing precision for all fake samples .....
..... beta radius for fake ball is 0.0185 .....
..... computing recall for all fake samples .....
..... computing authenticity for all fake samples .....


In [100]:
print(P.mean())

0.6516634050880626


In [101]:
print(R.mean())

0.0


In [102]:
print(A.mean())

0.9980430528375733


In [112]:
C

0.0

In [109]:
def make_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    else:
        return x
    
    

def one_class_SVM_loss(embedded_x, center, radius, nu):
    r"""Compute One class SVM loss for data points x.
    ..math:
        L = \sum(l_i)
        l_i = r^2 + (1/nu) * max(0, ||x_i - center||^2 - r^2)
    """
    # for each sample compute L2 distance to center
    n = embedded_x.shape[0] # num samples
    l1 = torch.pow(embedded_x - center, 2).sum(dim=-1)
    l2 = nn.functional.relu(l1 - radius**2)
    
    return (radius**2 + (1/nu)*l2).sum()


pdist = nn.functional.pairwise_distance


def compute_quantile_radius(x, center, alpha=0.95):
    """Given a set of data points in embedded / data space and a center
        returns: 
    """ 
    # pairwise distances between each point x and center 
    D = pdist(x, center)
    
    return torch.quantile(D, alpha)
    
    
def precision_classifier(fake_x, center, r_alpha):
    """Precision classifier tests if the fake sample 'fake_x'
        outside of the alpha ball around reals B(center_r, r_alpha)
    """
    pred = pdist(fake_x.view(1,-1), center.view(1,-1)) <= r_alpha
    return float(pred.detach())

def knn(x, K=10):
    """Make a sklearn.neighbors object for datapoints in tensor x
        x has shape (N, D), N = num points
    """
    x = make_numpy(x)
    
    return NearestNeighbors(n_neighbors=K, algorithm='ball_tree').fit(x)
    
    
    
def recall_classifier(real_x, fake_x_pts, nearest_neighbors, r_beta=None, K=10, beta=0.95):
    """
    1. Make a Ball(center_g, radius_beta). To do so, use the fake_x_pts 
        to get the quantile radius, eliminate fake pts outside the quantile radius.
    2. Find the nearest neighbor distance of real_x to all other reals
    3. Find the nearest fake to real_x, lying in the ball B(c_g, r_beta)
    4. Return : Is distance between nearest_fake and real_x <= NND ? 
    
    """
    # make center the average fake data
    c_g = fake_x_pts.mean(dim=0).view(1,-1)
    
    if r_beta is None:
        # get ball B radius
        r_beta = compute_quantile_radius(fake_x_pts, c_g, beta)
        
    # which pts are in the ball ? 
    in_ball = pdist(fake_x_pts, c_g) <= r_beta
    
    # find the K nearest neighbors of real_x in all other reals
    rx = make_numpy(real_x)
    dists, _ = nearest_neighbors.kneighbors(rx.reshape(1,-1), K)
    NND = dists.squeeze()[-1] # Kth nearest neighbor
    
    # find nearest pt to real_x in ball B(c_g, r_beta)
    D = pdist(fake_x_pts[in_ball], real_x)
    nearest_fake = torch.argmin(D)
    
    # is nearest fake in Ball(real_x, NND) ? 
    pred = pdist(fake_x_pts[nearest_fake].view(1,-1), real_x.view(1,-1)) <= NND
    return float(pred.detach())
    

def authenticity_classifier(fake_x, real_x_pts):
    """
    """
    # find distance between fake pt and all reals
    D = pdist(fake_x, real_x_pts)
    # find nearest real to fake
    i = torch.argmin(D) # i* in paper
    d_g_j = torch.min(D) # d_{g,j} in paper
    
    real_x_pts_wo = torch.cat([real_x_pts[:i],real_x_pts[i+1:]],dim=0)
    
    d_r_i = pdist(real_x_pts[i], real_x_pts_wo).min() # smallest distance to rest
    
    # output of authenticity classifier is A_j=1 (authentic) if the 
    # distance d_r_i < d_g_j
    pred = d_r_i < d_g_j
    return float(pred.detach())
        

def density_and_coverage(fake_x, real_x_pts, K=10):
    """From Naeem et al 2020.
        Density is defined as (1/kM)*(1/n) \sum_j \sum_i (Is fake_j  in ball B(x_i, NND_k(x_i))
    """
    nearest_neighbors = knn(real_x_pts, K)
    
    D = []
    C = np.zeros(len(real_x_pts))
    M = fake_x.shape[0]
    for i in range(real_x_pts.shape[1]):
        
        for j in range(M):
            
            # is Y_j in Ball around X_i ?
            # what is the Ball around X_i
            rx = make_numpy(real_x_pts[i])
            dists, _ = nearest_neighbors.kneighbors(rx.reshape(1,-1), n_neighbors=K,
                                                         return_distance=True)
            # find the Kth neighbor distance
            NND = dists.squeeze()[-1]
            
            pred = pdist(fake_x[j].view(1,-1), real_x_pts[i].view(1,-1)) <= NND
            D.append(float(pred.detach()))
            
            # for coverage
            if D[-1] and C[i] == 0.:
                C[i] = 1.
            
    D = np.sum(D)
    return (1/K)*(1/M)*D, C.mean()
            
            
    
def learn_embedding(data, opts):
    print('..... making tensor dataset and dataloader .....')
    traindataset = make_dataset(data, opts['max_length'])
    traindataloader = DataLoader(traindataset, batch_size=opts['batch_size'], shuffle=True, drop_last=True)
    
    print('..... training embedding .....')
    embedder = EmbeddingNet(data[0].shape[0], opts['nembed'], nrnn=opts['nrnn'], nrnnlayers=opts['nrnnlayers'],
                       dropout=opts['dropout'], nlin=opts['nlin'])
    if opts['cuda']:
        embedder = embedder.cuda()

    return train(embedder, traindataloader, opts)


def embed_list_of_sequences(embedder, seqs, cuda=True):
    Xtilde = []
    for x in seqs:
        x = torch.from_numpy(x).to(torch.float32)
        if cuda:
            x = x.cuda()
        Xtilde.append(embedder(x.view(1, x.shape[0], x.shape[1])))
    return torch.cat(Xtilde, dim=0)
    

In [107]:
def generate_samples(netG, hmm, nsamples=1, invtemp=1., timesteps=[], cuda=True):
    """Generate samples from trained netG and hmm"""
    seqs = [tempered_sampling(hmm, invtemp, timesteps=timesteps[i], 
                                sample_obs=True, start_state_max=True, 
                                 sample_var = 0.)[0] for i in range(nsamples)]
    # decode with netG
    seqs_out = [None for _ in range(len(seqs))]
    for i in range(len(seqs)):
        seqs_out[i] = overlap_decode(seqs[i], netG,  noverlap = 0,
                                          cuda = cuda, get_audio = False)[0]
    
    return seqs_out
